---
title: "Experimento con multicolinealidad"
description: "Experimentos con multicolinealidad para ver si siempre es tan mala como se dice en la regresión lineal"
description-meta: "Experimentos con multicolinealidad para ver si siempre es tan mala como se dice en la regresión lineal"
author: "Leonardo Hansa"
date: "2025-01-16"
categories: [datos]
execute: 
  echo: true
  message: false
  warning: false
freeze: true
---

In [ ]:
#| label: libs
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

## Multicolinealidad en la regresión lineal con muestras grandes

Tienes una variable $y$ que depende de dos variables $x_1$ y $x_2$. Pero $x_2$ es una combinación lineal de $x_1$. 


In [ ]:
#| label: datos

np.random.seed(123)

n = 10000
x1 = np.random.normal(50, 10, n)
x2 = 0.5 * x1 + np.random.normal(0, 5, n)
y = 2 + 3 * x1 + 4 * x2 + np.random.normal(0, 10, n)

Por si hay alguna duda:


In [ ]:
#| label: correlacion
np.corrcoef(x1, x2)[0, 1]

¿Qué pasa si usas $x_1$ y $x_2$ en una regresión lineal?


In [ ]:
#| label: regresion
X = np.column_stack((x1, x2))
X = sm.add_constant(X)

modelo = OLS(y, X).fit()
print(modelo.summary())

Pues aparentemente nada. Los coeficientes son los esperados y las $t$-stats son altas.

## Multicolinealidad en la regresión lineal con muestras pequeñas

Antes tenías una muestra de tamaño `{python} n`. Vamos a cambiar eso.


In [ ]:
#| label: datos-pequenos
n = 100
x1 = np.random.normal(50, 10, n)
x2 = 0.5 * x1 + np.random.normal(0, 5, n)
y = 2 + 3 * x1 + 4 * x2 + np.random.normal(0, 10, n)

Con estos datos, aparentemente los mismos pero con menor muestra, ajusto la regresión. 


In [ ]:
#| label: regresion-pequena
X = np.column_stack((x1, x2))
X = sm.add_constant(X)

modelo = OLS(y, X).fit()
print(modelo.summary())

Los coeficientes de $x_1$ y $x_2$ no son horribles, pero el intercept ya no tiene sentido. 

## Incertidumbre

Los resultados de la regresión pueden mejorar, no solo por tener más datos, sino simplemente por una muestra que encaje mejor. 

Así que vamos a generar muchas muestras, de distintos tamaños, y vemos en cada una cómo cambian los coeficientes.


In [ ]:
#| label: simulacion

n_sizes = np.array([100, 1000, 2500, 5000, 7500, 10000])

runs_per_size = 50

coeficientes = np.zeros((len(n_sizes), runs_per_size, 3))

for i, n in enumerate(n_sizes):
    for j in range(runs_per_size):
        x1 = np.random.normal(50, 10, n)
        x2 = 0.5 * x1 + np.random.normal(0, 5, n)
        y = 2 + 3 * x1 + 4 * x2 + np.random.normal(0, 10, n)
        
        X = np.column_stack((x1, x2))
        X = sm.add_constant(X)

        modelo = OLS(y, X).fit()
        coeficientes[i, j] = modelo.params

Visualizo los coeficientes como puntos, incluido el intercept. En el eje $x$ tienes el tamaño de la muestra y en el eje $y$ el valor del coeficiente. La línea horizontal es el valor real del coeficiente.

Fíjate cómo, cuanto menor es el tamaño muestral, menor es la precisión de la estimación. No creas que por que alguna vez salga bien con una muestra, tu estimación va a ser buena. La incertidumbre es alta, te guste o no.


In [ ]:
#| label: grafico

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize=(18, 4))

for i in range(runs_per_size):
    ax[0].scatter(n_sizes, coeficientes[:, i, 0], color='blue', alpha=0.1)
    ax[1].scatter(n_sizes, coeficientes[:, i, 1], color='blue', alpha=0.1)
    ax[2].scatter(n_sizes, coeficientes[:, i, 2], color='blue', alpha=0.1)

ax[0].axhline(2, color='red', linestyle='--')
ax[1].axhline(3, color='red', linestyle='--')
ax[2].axhline(4, color='red', linestyle='--')


ax[0].set_title('Intercept')
ax[1].set_title('Coeficiente de x1')
ax[2].set_title('Coeficiente de x2')

plt.show()